In [38]:
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
from neuromaps import datasets, transforms, images, resampling, nulls, stats

In [3]:
!wb_command -version

Connectome Workbench
Type: Command Line Application
Version: 2.0.1
Qt Compiled Version: 6.2.3
Qt Runtime Version: 6.2.3
Commit: 150de12f4f4b94b39bec6d9133ad2e7019d2d3ef
Commit Date: 2024-10-15 17:38:41 -0500
Compiler: c++ (/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin)
Compiler Version: 13.0.0.13000029
Compiled Debug: NO
Operating System: Apple OSX
Compiled with OpenMP: YES


In [20]:
brain_maps = [
    {'source': 'abagen', 'desc': 'genepc1', 'space': 'fsaverage', 'den': '10k'},
    {'source': 'hcps1200', 'desc': 'myelinmap', 'space': 'fsLR', 'den': '32k'},
    {'source': 'hcps1200', 'desc': 'thickness', 'space': 'fsLR', 'den': '32k'},
    {'source': 'reardon2018', 'desc': 'scalingnih', 'space': 'civet', 'den': '41k'},
    {'source': 'savli2012', 'desc': 'way100635', 'space': 'MNI152', 'res': '3mm'},
    {'source': 'margulies2016', 'desc': 'fcgradient01', 'space': 'fsLR', 'den': '32k'},
    {'source': 'raichle', 'desc': 'cmrglc', 'space': 'fsLR', 'den': '164k'},
    {'source': 'mueller2013', 'desc': 'intersubjvar', 'space': 'fsLR', 'den': '164k'}
]
single_sphere_counter = 0
for map in brain_maps:
    brain_map = datasets.fetch_annotation(**map)
    if len(brain_map) != 2:
        single_sphere_counter += 1
        print(map)
print(single_sphere_counter)


[References] Please cite the following papers if you are using this data:

  For {'source': 'abagen', 'desc': 'genepc1', 'space': 'fsaverage', 'den': '10k'}:
  [primary]:
    Michael J Hawrylycz, Ed S Lein, Angela L Guillozet-Bongaarts, Elaine H Shen, Lydia Ng, Jeremy A Miller, Louie N Van De Lagemaat, Kimberly A Smith, Amanda Ebbert, Zackery L Riley, and others. An anatomically comprehensive atlas of the adult human brain transcriptome. Nature, 489(7416):391, 2012.
    Ross D Markello, Aurina Arnatkeviciute, Jean-Baptiste Poline, Ben D Fulcher, Alex Fornito, and Bratislav Misic. Standardizing workflows in imaging transcriptomics with the abagen toolbox. eLife, 10:e72129, 2021.
  [secondary]:
    

[References] Please cite the following papers if you are using this data:

  For {'source': 'hcps1200', 'desc': 'myelinmap', 'space': 'fsLR', 'den': '32k'}:
  [primary]:
    Matthew F Glasser, Timothy S Coalson, Emma C Robinson, Carl D Hacker, John Harwell, Essa Yacoub, Kamil Ugurbil, Jespe

In [36]:
def is_volumetric(map_dict):
    """Check if a map is volumetric (MNI space)"""
    return map_dict.get('space') in ['MNI152', 'MNI305']


def load_and_prepare_map(map_dict, target_space='fsLR', target_den='164k'):
    """
    Load a brain map and prepare it for analysis
    - Volumetric maps: transform to surface
    - Surface maps: resample if needed, use both hemispheres
    - return (transformed data array, space, density)
    """
    print(f"  Loading {map_dict['desc']}...")
    
    # Fetch the map
    brain_map = datasets.fetch_annotation(**map_dict)
    
    # Check if volumetric
    if is_volumetric(map_dict):
        print(f"Volumetric map detected, transforming to {target_space} surface...")
        
        # Transform volumetric to surface (both hemispheres)
        try:
            surface_map = transforms.mni152_to_fslr(brain_map, fslr_density=target_den)
            # surface_map will be a tuple of (left_hemi, right_hemi)
            lh_data = images.load_data(surface_map[0])
            rh_data = images.load_data(surface_map[1])
            full_data = np.hstack([lh_data, rh_data])
            return full_data, target_space, target_den
            
        except Exception as e:
            print(f"    Warning: Could not transform volumetric map: {e}")
 
    # Surface map processing
    else:
        src_space = map_dict.get('space')
        src_den = map_dict.get('den')
        template = {'source': 'hcps1200', 'desc': 'thickness', 'space': 'fsLR', 'den': '32k'}
        template_map = datasets.fetch_annotation(**template)
        # Load both hemispheres
        if len(brain_map) == 2:
            lh_map, rh_map = brain_map
        else:
            raise ValueError(f"Expected 2 hemispheres for surface map, got {len(brain_map)}")
        
        # Resample if needed
        if src_den != target_den or src_space != target_space:
            print(f"Resampling from {src_space}-{src_den} to {target_space}-{target_den}...")            
            # Resample both hemispheres
            lh_resampled, _ = resampling.resample_images(
                lh_map, template_map[0],
                src_space=src_space, trg_space=target_space,
                hemi='L', resampling='transform_to_trg'
            )
            rh_resampled, _ = resampling.resample_images(
                rh_map, template_map[1],
                src_space=src_space, trg_space=target_space,
                hemi='R', resampling='transform_to_trg'
            )
            
            lh_data = images.load_data(lh_resampled)
            rh_data = images.load_data(rh_resampled)
        else:
            lh_data = images.load_data(lh_map)
            rh_data = images.load_data(rh_map)
        
        # Concatenate both hemispheres
        full_data = np.hstack([lh_data, rh_data])
        return full_data, src_space if src_den == target_den else target_space, \
               src_den if src_den == target_den else target_den


In [29]:
map = load_and_prepare_map(brain_maps[4])

  Loading way100635...

[References] Please cite the following papers if you are using this data:

  For {'source': 'savli2012', 'desc': 'way100635', 'space': 'MNI152', 'res': '3mm'}:
  [primary]:
    Markus Savli, Andreas Bauer, Markus Mitterhauser, Yu-Shin Ding, Andreas Hahn, Tina Kroll, Alexander Neumeister, Daniela Haeusler, Johanna Ungersboeck, Shannan Henry, and others. Normative database of the serotonergic system in healthy subjects using multi-tracer pet. Neuroimage, 63(1):447–459, 2012.
  [secondary]:
    
Volumetric map detected, transforming to fsLR surface...


In [39]:
def compute_spin_covariance_matrix(brain_maps, n_perm=1000, target_space='fsLR', 
                                   target_den='32k', seed=1234):
    """
    Compute covariance matrix with Alexander-Bloch spin test correction
    Uses BOTH hemispheres for all maps
    
    Returns:
        cov_empirical: Empirical covariance matrix
        cov_pvalues: P-values for each covariance
        cov_corrected: Covariance matrix with non-significant values set to 0
        correlation_matrix: Correlation matrix (standardized covariance)
    """
    n_maps = len(brain_maps)
    prepared_maps = []
    map_names = []
    map_spaces = []
    map_dens = []
    for i, brain_map in enumerate(brain_maps):
        print(f"\nMap {i+1}/{n_maps}: {brain_map['source']} - {brain_map['desc']}")
        try:
            prep_map, final_space, final_den = load_and_prepare_map(
                brain_map, target_space, target_den
            )
            prepared_maps.append(prep_map)
            map_names.append(f"{brain_map['source']}_{brain_map['desc']}")
            map_spaces.append(final_space)
            map_dens.append(final_den)
            print(f"Successfully prepared: {len(prep_map)} vertices")
        except Exception as e:
            print(f"Failed to prepare map: {e}")
            raise
    print("Generating spatial null distributions")
    nulls_list = []
    for i, prep_map in enumerate(prepared_maps):
        print(f"\nMap {i+1}/{n_maps}: {map_names[i]}")
        print(f"  Generating {n_perm} spin rotations...")
        try:
            rotated = nulls.alexander_bloch(
                prep_map,
                atlas=target_space,
                density=target_den,
                n_perm=n_perm,
                seed=seed  # Different seed for each map
            )
            nulls_list.append(rotated)
            print(f"Generated null distribution: shape {rotated.shape}")
        except Exception as e:
            print(f"Failed to generate nulls: {e}")
            raise

    cov_empirical = np.zeros((n_maps, n_maps))
    cov_pvalues = np.ones((n_maps, n_maps))
    corr_empirical = np.zeros((n_maps, n_maps))
    
    # Diagonal elements (variances)
    for i in range(n_maps):
        mask = ~np.isnan(prepared_maps[i])
        cov_empirical[i, i] = np.var(prepared_maps[i][mask])
        cov_pvalues[i, i] = 0.0
        corr_empirical[i, i] = 1.0
    
    # Off-diagonal elements (covariances)
    pair_count = 0
    total_pairs = n_maps * (n_maps - 1) // 2
    
    for i in range(n_maps):
        for j in range(i+1, n_maps):
            pair_count += 1
            print(f"\nPair {pair_count}/{total_pairs}: {map_names[i]} vs {map_names[j]}")
            
            # Check if maps have compatible dimensions
            if len(prepared_maps[i]) != len(prepared_maps[j]):
                print(f"Maps have different lengths: {len(prepared_maps[i])} vs {len(prepared_maps[j])}")
                print(f"Skipping this pair...")
                continue
            
            # Use compare_images to get correlation with spin test
            # This compares rotated versions of map i to ORIGINAL map j
            try:
                corr_emp, p_spin = stats.compare_images(
                    prepared_maps[i],  # source (will be rotated)
                    prepared_maps[j],  # target (stays original)
                    metric='pearsonr',
                    nulls=nulls_list[i],  # rotated versions of source
                    nan_policy='omit'
                )
                
                print(f"Correlation: {corr_emp:.4f}")
                print(f"P-value (spin): {p_spin:.4f} {'***' if p_spin < 0.001 else '**' if p_spin < 0.01 else '*' if p_spin < 0.05 else ''}")
                
                # Store correlation
                corr_empirical[i, j] = corr_emp
                corr_empirical[j, i] = corr_emp
                cov_pvalues[i, j] = p_spin
                cov_pvalues[j, i] = p_spin
                
                # Compute covariance from correlation
                # Cov(X,Y) = Corr(X,Y) * SD(X) * SD(Y)
                mask = ~np.isnan(prepared_maps[i]) & ~np.isnan(prepared_maps[j])
                std_i = np.std(prepared_maps[i][mask])
                std_j = np.std(prepared_maps[j][mask])
                cov_emp = corr_emp * std_i * std_j
                
                cov_empirical[i, j] = cov_emp
                cov_empirical[j, i] = cov_emp
                
                print(f"Covariance: {cov_emp:.6f}")
                print(f"Valid vertices: {np.sum(mask)}/{len(mask)}")
                
            except Exception as e:
                print(f"Failed to compare maps: {e}")
                continue
    
    # Create corrected matrices (only keep significant results)
    cov_corrected = cov_empirical.copy()
    cov_corrected[cov_pvalues > 0.05] = 0
    np.fill_diagonal(cov_corrected, np.diag(cov_empirical))
    
    corr_corrected = corr_empirical.copy()
    corr_corrected[cov_pvalues > 0.05] = 0
    np.fill_diagonal(corr_corrected, 1.0)
    
    corr_corrected = corr_empirical.copy()
    corr_corrected[cov_pvalues > 0.05] = 0
    np.fill_diagonal(corr_corrected, 1.0)
    
    return cov_empirical, cov_pvalues, cov_corrected, corr_empirical, corr_corrected, map_names

In [40]:
import time
start_time = time.perf_counter()

cov_emp, cov_pvals, cov_corr, corr_emp, corr_corr, names = compute_spin_covariance_matrix(
    brain_maps, 
    n_perm=1000,
    target_space='fsLR',
    target_den='32k',  # Using 32k as it's a good middle ground
    seed=1234
)

end_time = time.perf_counter()
cov_corr_df = pd.DataFrame(cov_corr, index=names, columns=names)
print(cov_corr_df)



Map 1/8: abagen - genepc1
  Loading genepc1...

[References] Please cite the following papers if you are using this data:

  For {'source': 'abagen', 'desc': 'genepc1', 'space': 'fsaverage', 'den': '10k'}:
  [primary]:
    Michael J Hawrylycz, Ed S Lein, Angela L Guillozet-Bongaarts, Elaine H Shen, Lydia Ng, Jeremy A Miller, Louie N Van De Lagemaat, Kimberly A Smith, Amanda Ebbert, Zackery L Riley, and others. An anatomically comprehensive atlas of the adult human brain transcriptome. Nature, 489(7416):391, 2012.
    Ross D Markello, Aurina Arnatkeviciute, Jean-Baptiste Poline, Ben D Fulcher, Alex Fornito, and Bratislav Misic. Standardizing workflows in imaging transcriptomics with the abagen toolbox. eLife, 10:e72129, 2021.
  [secondary]:
    

[References] Please cite the following papers if you are using this data:

  For {'source': 'hcps1200', 'desc': 'thickness', 'space': 'fsLR', 'den': '32k'}:
  [primary]:
    Matthew F Glasser, Timothy S Coalson, Emma C Robinson, Carl D Hacker,

In [41]:
cov_corr_df

,abagen_genepc1,hcps1200_myelinmap,hcps1200_thickness,reardon2018_scalingnih,savli2012_way100635,margulies2016_fcgradient01,raichle_cmrglc,mueller2013_intersubjvar
abagen_genepc1,0.879407,0.291597,-0.535008,0.000000,-8.351730,-1.942930,8.039151e+02,-0.071440
hcps1200_myelinmap,0.291597,0.155926,-0.180156,0.000000,-3.204581,-0.821157,0.000000e+00,-0.032289
hcps1200_thickness,-0.535008,-0.180156,0.669394,0.000000,6.551888,1.158190,-6.284991e+02,0.000000
reardon2018_scalingnih,0.000000,0.000000,0.000000,0.114502,0.000000,0.000000,0.000000e+00,0.021048
savli2012_way100635,-8.351730,-3.204581,6.551888,0.000000,162.615509,19.215811,-1.308449e+04,0.753412
margulies2016_fcgradient01,-1.942930,-0.821157,1.158190,0.000000,19.215811,15.770119,0.000000e+00,0.336251
raichle_cmrglc,803.915100,0.000000,-628.499084,0.000000,-13084.491211,0.000000,3.728586e+06,0.000000
mueller2013_intersubjvar,-0.071440,-0.032289,0.000000,0.021048,0.753412,0.336251,0.000000e+00,0.030165
